# Benchmark for Community detection

In [1]:
"""Load/import helper functions"""

import time
import random
from LocalPopular import locally_popular_clustering_with_hop_distance, extract_labels_from_communities, time_tester, calculate_scores_CD

from GraphFunctions import generate_agents, calculate_euclidian_relationships, create_graph, \
    my_make_circles, create_graphs_euclid, create_graphs_kNN, \
    generate_graph,create_graphs_hop_distance, create_graphs_hop_distance_abs,randomize_graph_node_labels

from PlotHelperFunctions import plot_clustering, plot_stuff

from sklearn.cluster import KMeans, DBSCAN
from sklearn.datasets import make_moons
from sklearn.metrics import rand_score
import numpy as np
import networkx as nx
from scipy.spatial import distance

from community_detection.leiden import leiden
from community_detection.louvain import louvain
from community_detection.quality_functions import CPM, Modularity


import data.jazz as jazz
import data.cora as cora

## Create Graphs

In [2]:
repetitions = 1    #Number of random isomorph permutation for each graph

cora_graph = cora.get_graph()

cora_graph = nx.relabel_nodes(cora_graph, {list(cora_graph.nodes())[i] : i for i in range(len(cora_graph.nodes()))} )
cora_truth = list(cora_graph.nodes[i]['subject'] for i in range(len(cora_graph.nodes())))
cora_perm_graph = []
cora_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(cora_graph,cora_truth)
    cora_perm_graph += [g]
    cora_perm_truth += [t]


jazz_graph = jazz.get_graph()
jazz_graph = nx.relabel_nodes(jazz_graph, {i : i-1 for i in range(len(jazz_graph)+1)} )
jazz_graph,_ = randomize_graph_node_labels(jazz_graph,None)
jazz_truth = None

jazz_perm_graph = []
jazz_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(jazz_graph,jazz_truth)
    jazz_perm_graph += [g]
    jazz_perm_truth += [t]


karate_graph = nx.karate_club_graph()
#karate_graph,_ = randomize_graph_node_labels(karate_graph,None)
karate_truth = list(karate_graph.nodes[i]["club"] for i in range(34))

karate_perm_graph = []
karate_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(karate_graph,karate_truth)
    karate_perm_graph += [g]
    karate_perm_truth += [t]

karate_perm_graph = [karate_graph,karate_graph,karate_graph,karate_graph,karate_graph]
karate_perm_truth = [karate_truth,karate_truth,karate_truth,karate_truth,karate_truth]

graph,graph_truth = generate_graph(10,25,0.2,0.05)

graph_perm_graph = []
graph_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(graph,graph_truth)
    graph_perm_graph += [g]
    graph_perm_truth += [t]




## Run the algorithms


In [3]:
import itertools
import timeit

f = 0.2   #f-bound
e = 0.2   #e-bound


graphs = [karate_perm_graph,cora_perm_graph,jazz_perm_graph,graph_perm_graph]
expected_clusters = [2,7,None,25]
graph_names = [ 'Karate Club','Cora','Jazz','25 quasi cliques (Name WIP)']
graph_truths =  [karate_perm_truth,cora_perm_truth,jazz_perm_truth,graph_perm_truth]


𝓗 = Modularity(1.0)

fn_louvain_mod = lambda G,_: louvain(G, 𝓗)
fn_leiden_mod  = lambda G,_: leiden(G, 𝓗)

louv_out = None
lei_out = None

algorithms = [ fn_louvain_mod, fn_leiden_mod]
algo_names = [ 'Louvain (Mod)', 'Leiden (Mod)']

lp_a_b =lambda agents, initial_clustering, pre: locally_popular_clustering_with_hop_distance(agents, f, e, initial_clustering,mode='B',pre = pre)
lp_a_f =lambda agents, initial_clustering, pre: locally_popular_clustering_with_hop_distance(agents, f, e, initial_clustering,mode='F',pre = pre)
lp_a_e =lambda agents, initial_clustering, pre: locally_popular_clustering_with_hop_distance(agents, f, e, initial_clustering,mode='E',pre = pre)

algorithms = [ fn_louvain_mod, fn_leiden_mod,lp_a_b,lp_a_f,lp_a_e]
algo_names = [ 'Louvain (Mod)', 'Leiden (Mod)','LP (Balanced) Heuristic',\
               'LP (Friend-Oriented) Heuristic','LP (Enemy-Averse) Heuristic']
is_lp_heuristic = [False,False,True, True, True]

collected_data = {}
for ((graph, g_name,clusters,truth), (algo, a_name,lp_heuristic)) in \
    itertools.product(zip(graphs, graph_names, expected_clusters,graph_truths), zip(algorithms, algo_names,is_lp_heuristic)):
    
        
        
    agents = []
    for i in range(len(graph)):
        agents += [list(graph[i].nodes())]


    if lp_heuristic:
        # start with everyone alone
        a_name_modified = a_name + ' starting with everyone alone'
        initial_clusters = len(agents[0])
        if graph == cora_perm_graph:
            a_name_modified += ' *6 starting clusters'
            initial_clusters = 6
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        
        test_callable = lambda a: algo(a,initial_clusters,None)
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

        
        # starting with predicted number of clusters
        a_name_modified = a_name + ' starting with predicted number of clusters'
        initial_clusters = clusters
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        
        test_callable = lambda a: algo(a,initial_clusters,None)
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

        
        # start with the output of leiden
        a_name_modified = a_name + ' starting with the output of leiden'
        initial_clusters = clusters
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        
        test_callable = lambda a: algo(a,initial_clusters,fn_leiden_mod)
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)

        rand_score_with_init = sum(rand_score(list(out.values()), list(lei.values())) for out, lei in zip(outputs, lei_output)) / len(outputs)
        scores['Rand Score with initial clustering'] = rand_score_with_init
        
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

       

    else:
        print(f"Running {a_name} on {g_name} … ", end='')
        test_callable = lambda a : algo(a,_)
        times,outputs = time_tester(test_callable,graph)
        outputs = [extract_labels_from_communities(c.communities) for c in outputs]

        if algo == fn_leiden_mod:
            lei_output = outputs
            
        
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)
        scores['Time'] = avg_time
        collected_data[(a_name,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))


Running Louvain (Mod) on Karate Club … execution time: ~ 0.005626 seconds.
Rand Index : ~ 0.6762923351158645
Modularity : ~ 0.4377148104420832
Time : ~ 0.005625540018081665
Running Leiden (Mod) on Karate Club … execution time: ~ 0.010929 seconds.
Rand Index : ~ 0.6502673796791444
Modularity : ~ 0.44077134986225897
Time : ~ 0.010928939981386065
Running LP (Balanced) Heuristic starting with everyone alone on Karate Club … execution time: ~ 0.016650 seconds.
Rand Index : ~ 0.5525846702317291
Modularity : ~ 0.2066115702479338
Time : ~ 0.016649559931829573
Running LP (Balanced) Heuristic starting with predicted number of clusters on Karate Club … execution time: ~ 0.001535 seconds.
Rand Index : ~ 0.5294117647058824
Modularity : ~ 0.2790708569929349
Time : ~ 0.0015352800488471984
Running LP (Balanced) Heuristic starting with the output of leiden on Karate Club … execution time: ~ 0.012319 seconds.
Rand Index : ~ 0.6998217468805705
Modularity : ~ 0.4272202544929817
Rand Score with initial clu

## Gather the numbers

We can use the collected_data dictionairy to build a table for better comparison


In [4]:
print(collected_data)

{('Louvain (Mod)', 'Karate Club'): {'Rand Index': np.float64(0.6762923351158645), 'Modularity': 0.4377148104420832, 'Time': 0.005625540018081665}, ('Leiden (Mod)', 'Karate Club'): {'Rand Index': np.float64(0.6502673796791444), 'Modularity': 0.44077134986225897, 'Time': 0.010928939981386065}, ('LP (Balanced) Heuristic starting with everyone alone', 'Karate Club'): {'Rand Index': np.float64(0.5525846702317291), 'Modularity': 0.2066115702479338, 'Time': 0.016649559931829573}, ('LP (Balanced) Heuristic starting with predicted number of clusters', 'Karate Club'): {'Rand Index': np.float64(0.5294117647058824), 'Modularity': 0.2790708569929349, 'Time': 0.0015352800488471984}, ('LP (Balanced) Heuristic starting with the output of leiden', 'Karate Club'): {'Rand Index': np.float64(0.6998217468805705), 'Modularity': 0.4272202544929817, 'Rand Score with initial clustering': np.float64(0.6944741532976828), 'Time': 0.01231892006471753}, ('LP (Friend-Oriented) Heuristic starting with everyone alone'